In [1]:
import jsonlines, json
from nltk.stem import PorterStemmer

path = ["data/inverted_index_1.jsonl", "data/inverted_index_2.jsonl"]
inverted_index = {}
url_map = {}

ModuleNotFoundError: No module named 'jsonlines'

In [10]:
# read the jsonl file to save the inverted index to a map
with jsonlines.open(path[0], 'r') as file:
    for jsonObj in file:
        token = list(jsonObj.keys())[0]
        inverted_index[token] = jsonObj[token]

In [11]:
# read the jsonl files to save the inverted index to a map
with jsonlines.open(path[1], 'r') as file:
    for jsonObj in file:
        token = list(jsonObj.keys())[0]
        inverted_index[token] = jsonObj[token]

In [5]:
# for debugging purpose
# i = 0
# for key in inverted_index:
#     print(key, inverted_index[key])
#     i += 1

#     if i > 10: break

In [12]:
# read the json file to save the urls' id to a map
with open("url_id.json", 'r') as file:
    data = json.load(file)
    for id in data:
        url_map[id] = data[id]

In [7]:
# for debugging purpose 
# i = 0
# for key in url_map:
#     print(key, url_map[key])
#     i += 1

#     if i > 10: break

In [13]:
while True:

    posting_lists = []      # for saving all the url id 
    searching_tokens = [] 
    
    # ask for input/quries
    queries = input("Input queries (type exit to stop searching): ")

    if queries.lower() == "exit":
        break

    # perform the search
    # cristina lopes, machine learning, ACM, master of software engineering

    searching_tokens = queries.split()  # ex: "machine learning" => ['machine', 'learning']

    # note: need a stemming step here before start searching

    stemmer = PorterStemmer()
    stemmed_searching_tokens = [stemmer.stem(word) for word in searching_tokens]

    # get the posting list from each token and save to posting_lists array
    for token in stemmed_searching_tokens:
        if token in inverted_index:
            posting_lists.append(inverted_index[token])
        else:
            print(token, "is not found")

    # find the intersection of the posting lists
    url_id_set = set()
    for posting_list in posting_lists:
        if len(url_id_set) == 0:
            url_id_set.update(set(posting_list.keys()))
        else:
            # url_id_set.intersection(set(posting_list.keys()))
            url_id_set &= set(posting_list.keys())

    # output the urls
    i = 0
    print("URLs that contain", queries)
    for id in url_id_set:
        if i > 5: break
        print(url_map[id])
        i += 1
    print()


lopes is not found
URLs that contain cristina lopes
https://www.ics.uci.edu/faculty/faculty_dept?department=Informatics
https://www.informatics.uci.edu/explore/faculty-profiles/sam-malek/#content
https://www.ics.uci.edu/community/news/spotlight/spotlight_microsoftspellerchallenge.php
https://www.informatics.uci.edu/explore/faculty-profiles/
https://www.ics.uci.edu/community/news/view_news.php?id=1207
http://flamingo.ics.uci.edu/releases/4.0/src/filtertree/data/female_names.txt

machine is not found
learning is not found
URLs that contain machine learning

URLs that contain acm
https://wics.ics.uci.edu/event/ics-banquet/
https://grape.ics.uci.edu/wiki/public/wiki/cs222-2017-fall?version=61
https://grape.ics.uci.edu/wiki/public/wiki/cs222-2017-fall?version=63
https://wics.ics.uci.edu/event/week-4-homework-and-interview-help/
https://grape.ics.uci.edu/wiki/public/wiki/cs222-2019-fall?version=11
https://isg.ics.uci.edu/event/michal-shmueli-scheuer-conversational-bots-for-customer-support/


In [3]:
!pip install nltk jsonlines

from nltk.stem import PorterStemmer
import jsonlines, json
import math

N = 55393 # total number of documents
stemmer = PorterStemmer()

# Load inverted index
path = ["data/inverted_index_1.jsonl", "data/inverted_index_2.jsonl"]
inverted_index = {}

for p in path:
    with jsonlines.open(p, 'r') as file:
        for jsonObj in file:
            token = list(jsonObj.keys())[0]
            inverted_index[token] = jsonObj[token]

# Load URL map
url_map = {}
with open("url_id.json", 'r') as file:
    data = json.load(file)
    for id in data:
        url_map[id] = data[id]

# Query loop
while True:
    queries = input("Input queries (type exit to stop searching): ")
    if queries.lower() == "exit":
        break

    tokens = queries.split()
    stemmed_tokens = [stemmer.stem(t) for t in tokens]

    posting_lists = []
    for token in stemmed_tokens:
        if token in inverted_index:
            posting_lists.append(inverted_index[token])
        else:
            print(token, "is not found")

    # Boolean AND: intersect all posting lists
    url_id_set = set()
    for posting_list in posting_lists:
        doc_ids = set(posting_list.keys())
        if not url_id_set:
            url_id_set = doc_ids
        else:
            url_id_set &= doc_ids

    # compute tf-idf scores
    doc_scores = {}

    for doc_id in url_id_set:
        score = 0.0
        for token in stemmed_tokens:
            posting = inverted_index.get(token, {})
            tf = posting.get(doc_id, 0)
            df = len(posting) if posting else 1
            idf = math.log(N / (1 + df))
            score += tf * idf
        doc_scores[doc_id] = score

    ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)

    # Output up to 5 URLs (no ranking yet)
    print(f"\nURLs that contain: \"{queries}\"")
    for i, (doc_id, score) in enumerate(ranked_docs[:5]):
        url = url_map.get(doc_id, doc_id)
        print(f"{i+1}. {url} (tf-idf: {score:.4f})")
    print()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 17.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Input queries (type exit to stop searching):  cristina lopes



URLs that contain: "cristina lopes"
1. https://www.ics.uci.edu/~hsajnani/ (tf-idf: 230.9507)
2. https://www.ics.uci.edu/~sjavanma/ (tf-idf: 101.9723)
3. http://mondego.ics.uci.edu/ (tf-idf: 79.6436)
4. https://www.informatics.uci.edu/explore/facts-figures/#content (tf-idf: 74.4644)
5. https://www.informatics.uci.edu/explore/facts-figures/ (tf-idf: 74.4644)



Input queries (type exit to stop searching):  machine learning



URLs that contain: "machine learning"
1. http://mondego.ics.uci.edu/datasets/maven-contents.txt (tf-idf: 553.5183)
2. https://cml.ics.uci.edu/category/aiml/page/2/ (tf-idf: 472.7834)
3. https://cml.ics.uci.edu/category/aiml/page/2/#content (tf-idf: 472.7834)
4. https://cml.ics.uci.edu/category/aiml/ (tf-idf: 407.9766)
5. https://cml.ics.uci.edu/category/aiml/#content (tf-idf: 407.9766)



Input queries (type exit to stop searching):  acm



URLs that contain: "acm"
1. https://www.ics.uci.edu/~eppstein/pubs/pubs.ff (tf-idf: 421.6850)
2. https://www.ics.uci.edu/~eppstein/pubs/all.html (tf-idf: 389.0384)
3. https://www.ics.uci.edu/~kay/courses/131/s04readings.html (tf-idf: 375.4357)
4. https://www.ics.uci.edu/~eppstein/bibs/eppstein.html (tf-idf: 334.6274)
5. https://www.ics.uci.edu/~eppstein/pubs/geom-all.html (tf-idf: 263.8932)



Input queries (type exit to stop searching):  master of software engineering



URLs that contain: "master of software engineering"
1. https://www.ics.uci.edu/~kay/courses/h22/hw/DVD.txt (tf-idf: 1897.6876)
2. https://www.ics.uci.edu/~kay/courses/h22/hw/DVD-random.txt (tf-idf: 1897.6876)
3. https://www.ics.uci.edu/~eppstein/bibs/kpath.bib (tf-idf: 822.2951)
4. https://www.ics.uci.edu/~gbowker/classification/ (tf-idf: 713.0990)
5. https://www.ics.uci.edu/~dan/class/267P/datasets/calgary/news (tf-idf: 488.2932)



Input queries (type exit to stop searching):  exit
